In [3]:
# install required moduls
!pip install tensorflow tf2onnx

In [12]:
import numpy as np
import pandas as pd
import os
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
import tensorflow as tf
import tf2onnx
import onnx
import boto3


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
bucket = 'mstoffel'
subfolder = 'mstoffel/t15264971/measurements_c8y_Acceleration/'

In [6]:
##
# something along the line ...
##
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)
parquet_files = [];
for obj in my_bucket.objects.filter(Prefix=subfolder):
     if obj.key.endswith('.parquet'):
        parquet_files.append(obj)
        
parquet_files 

NameError: name 'boto3' is not defined

In [ ]:
##
# only use the 1. parquet file
##
parquet_file_1 = parquet_files[0]

s3 = s3fs.S3FileSystem(
    anon=False,
    use_ssl=True,
    client_kwargs={
        "region_name": 'eu-central-1',
        "verify": True,
    }
)



pf = pq.ParquetDataset('s3://' + bucket + '/' + parquet_file_1.key, filesystem=s3)
pf


In [ ]:
# show schema
pf.schema

In [ ]:
# convert parquet datatset to pandas
data = pf.read().to_pandas()
data.sample(5)

In [ ]:
#remove time column from the source
data.pop(data.columns[0])
data.sample(5)

In [ ]:

# drop unused column and rename colum
data = data.drop( ['creationTime', 'creationTimeOffset','creationTimeWithOffset', 'YEAR', 'MONTH', 'DAY', 'time', 'timeOffset', 'timeWithOffset', 'source', 'type',  'c8y_Acceleration.accelerationX.label', 'c8y_Acceleration.accelerationY.label'  ], axis = 1)

data = data.rename(columns={"c8y_Acceleration.accelerationZ.label": "label" ,"c8y_Acceleration.accelerationX.value": "accelerationX", "c8y_Acceleration.accelerationY.value": "accelerationY","c8y_Acceleration.accelerationZ.value": "accelerationZ"})

data.sample(5)

# pop the label columns
#labels = data.pop("label")
#print(labels)

In [ ]:
# pop the label columns
labels = data.pop("label")
print(labels)

In [ ]:
# normalize data
data = (data - data.mean()) / data.std() 
data.sample(5)

In [ ]:
# Reshape data to 3D array
data = data.values.reshape((-1, 3, 1))
print(data)

In [ ]:
# Convert labels to binary
labels = np.where(labels=="moving", 1, 0)

In [ ]:
# Split data into train and test sets
split = int(len(data) * 0.8)
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]

In [ ]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3, 1)),
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Reshape((3, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

In [ ]:
# Use the model to predict label for a single row of data
single_row = np.array([[[0.070770263671875,0.7673492431640625,0.751739501953125]]]).reshape((1, 3, 1))
#single_row = np.array([1.0,1.0,1.0]).reshape((1, 3, 1))
prediction = model.predict(single_row)
prediction 

In [ ]:
#MODEL_DIR = tempfile.gettempdir()
version = 1
#export_path = os.path.join(MODEL_DIR, str(version))
#print('export_path = {}\n'.format(export_path))

export_path = '../model/cumulocity_classification_model'

# Export the model as a SavedModel
tf.saved_model.save(model, export_path)


print('\nSaved model:' + export_path)
#!ls -l {export_path}

In [ ]:
local_model_path = f"{export_path}/model.onnx"
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, local_model_path)

In [ ]:
model_sub_folder = 'mstoffel/t15264971/model'
file_name = 'acceleration_model.onnx'

s3_model_path = f"{bucket}/{model_sub_folder}/{file_name}"

s3.put(local_model_path, s3_model_path)



In [11]:
# Create MXNetModel Sagemaker Model
from sagemaker.mxnet import MXNetModel
from sagemaker import get_execution_role
import tarfile

role = get_execution_role()


!pygmentize resnet152.py

with tarfile.open("onnx_model.tar.gz", mode="w:gz") as archive:
    archive.add(local_model_path)


bucket = session.Session().default_bucket()


Error: cannot read infile: [Errno 2] No such file or directory: 'resnet152.py'


NameError: name 'local_model_path' is not defined